In [1]:
from mxnet import nd
import random
import zipfile
import collections
import d2lzh as d2l
import math
from mxnet import autograd,gluon,nd
from mxnet.gluon import data as gdata,loss as gloss,nn
import random
import sys
import time
import zipfile
import numpy as np

In [2]:
def read_file_data(filePath):
    f=open(filePath)
    lines=f.readlines()
    raw_dataset=[st.split() for st in lines]
    wordNum=0;
    for st in raw_dataset:
         for wt in st:
            wordNum+=1
    #input_matrix=nd.array(raw_dataset)
    counter=collections.Counter([tk for st in raw_dataset for tk in st])
    idx_to_token=[tk for tk,_ in counter.items()]
    token_to_idx={tk:idx for idx, tk in enumerate(idx_to_token)}
    dataset=[[[token_to_idx[tk] for tk in st if tk in token_to_idx]] for st in raw_dataset]
    dict_size=800
    dataset=np.array(dataset)
    temp=np.array([])
    for row in dataset:
        for col in row:
            temp=np.append(temp,np.array(col))
    zeros=np.zeros(dict_size-wordNum)
    temp=np.append(temp,zeros)
    temp=nd.array(temp)
    temp.shape
    temp=temp.reshape(800,-1)
    return temp,wordNum

In [3]:
def conv_activation(w,b,z):
    sum=0;
    for t in z:
        sum+=t.sum()
    if sum==0:
        return 0
    temp=nd.dot(w,z)
    sum_two=temp+b
    for i in range(sum_two.shape[0]):
        for j in range(sum_two.shape[1]):
            sum_two[i][j]=max(sum_two[i][j],0)
    return sum_two

In [4]:
def corr2d(X,windows_size,kernel_w,kernel_h,param_w,param_b):
    Y=nd.zeros((X.shape[0],windows_size))
    for i in range(X.shape[0]):
        if(i+windows_size>=X.shape[0]):
            break
        Y[i]=conv_activation(param_w[i].reshape(1,-1),param_b[i],X[i:i+windows_size].T)
    return Y

In [5]:
def init_params():
    conv_one_W=nd.zeros((800,4))
    conv_one_W=nd.random.normal(scale=0.01,shape=(conv_one_W.shape))
    conv_one_W.reshape(800,4)
    conv_one_B=nd.zeros((800,1))
    conv_one_B=nd.random.normal(scale=0.01,shape=(conv_one_B.shape))

    conv_two_W=nd.zeros((400,3))
    conv_two_W=nd.random.normal(scale=0.01,shape=(conv_two_W.shape))
    conv_two_W.reshape(400,3)
    conv_two_B=nd.zeros((400,1))
    conv_two_B=nd.random.normal(scale=0.01,shape=(conv_two_B.shape))

    conv_three_W=nd.zeros((200,3))
    conv_three_W=nd.random.normal(scale=0.01,shape=(conv_three_W.shape))
    conv_three_B=nd.zeros((200,1))
    conv_three_B=nd.random.normal(scale=0.01,shape=(conv_three_B.shape))

    conv_four_W=nd.zeros((100,3))
    conv_four_W=nd.random.normal(scale=0.01,shape=(conv_four_W.shape))
    conv_four_B=nd.zeros((100,1))
    conv_four_B=nd.random.normal(scale=0.01,shape=(conv_four_B.shape))

    ful_connect_W=nd.zeros((100,80))
    ful_connect_W=nd.random.normal(scale=0.01,shape=(ful_connect_W.shape))
    ful_connect_B=nd.zeros((80,1))
    ful_connect_B=nd.random.normal(scale=0.01,shape=(ful_connect_B.shape))
    params=[conv_one_W,conv_one_B,conv_two_W,conv_two_B,conv_three_W,conv_three_B,conv_four_W,conv_four_B,ful_connect_W,ful_connect_B]
    return params

In [6]:
def myMax(X_one,X_two):
    sum_one=X_one.sum()
    sum_two=X_two.sum()
    if sum_one>sum_two:
        return X_one
    else :
        return X_two

In [7]:
def max_pool(X,width):
    temp= int(X.shape[0]/2)
    Y=nd.zeros((temp,width))
    j=0
    for i in range(0,X.shape[0]-1):
        if i%2==1:
            #print(j)
            Y[j]=myMax(X[i],X[i+1])
            j+=1
    return Y

In [8]:
def ful_connect_layer(X,W,B):
    Y=nd.zeros(W.shape[1])
    Y=nd.dot(X.T,W).T+B
    return Y.tanh()


In [9]:
def save_params(params,filePath):
    layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B=params
    file=filePath
    nd.save(file+'conv_layer_one_W.txt',layer_one_W)
    nd.save(file+'conv_layer_one_B.txt',layer_one_B)
    nd.save(file+'conv_layer_two_W.txt',layer_two_W)
    nd.save(file+'conv_layer_two_B.txt',layer_two_B)
    nd.save(file+'conv_layer_three_W.txt',layer_three_W)
    nd.save(file+'conv_layer_three_B.txt',layer_three_B)
    nd.save(file+'conv_layer_four_W.txt',layer_four_W)
    nd.save(file+'conv_layer_four_B.txt',layer_four_B)
    nd.save(file+'ful_layer_connect_W.txt',layer_connect_W)
    nd.save(file+'ful_layer_connect_B.txt',layer_connect_B)
    return

In [10]:
def read_params(filePath):
     layer_one_W=nd.load(filePath+'conv_layer_one_W.txt')
     layer_one_B=nd.load(filePath+'conv_layer_one_B.txt')
     layer_two_W=nd.load(filePath+'conv_layer_two_W.txt')
     layer_two_B=nd.load(filePath+'conv_layer_two_B.txt')
     layer_three_W=nd.load(filePath+'conv_layer_three_W.txt')
     layer_three_B=nd.load(filePath+'conv_layer_three_B.txt')
     layer_four_W=nd.load(filePath+'conv_layer_four_W.txt')
     layer_four_B=nd.load(filePath+'conv_layer_four_B.txt')
     layer_connect_W=nd.load(filePath+'ful_layer_connect_W.txt')
     layer_connect_B=nd.load(filePath+'ful_layer_connect_B.txt')
     params=[layer_one_W,layer_one_B,layer_two_W,layer_two_B,layer_three_W,layer_three_B,layer_four_W,layer_four_B,layer_connect_W,layer_connect_B]
     return params